#装载google云盘

In [ ]:
# %pip install google
# from google.colab import drive
# drive.mount('/content/drive')

#下载llama_factory

In [ ]:
%cd workspace
!git clone --depth 1 https://github.com/149285492/LLaMA-Factory.git
%cd LLaMA-Factory
%ls

%pip install -e .[torch,bitsandbytes]

import torch
try:
  assert torch.cuda.is_available() is True
except AssertionError:
  print("需要 GPU 环境")

可以自由修改 NAME 和 AUTHOR 变量的内容。

In [ ]:
import json


NAME = "小安"
AUTHOR = "HolmeHome"

with open("data/identity.json", "r", encoding="utf-8") as f:
  dataset = json.load(f)

for sample in dataset:
  sample["output"] = sample["output"].replace("{{"+ "name" + "}}", NAME).replace("{{"+ "author" + "}}", AUTHOR)

with open("data/identity.json", "w", encoding="utf-8") as f:
  json.dump(dataset, f, indent=2, ensure_ascii=False)

使用 LLaMA Board Web UI 微调模型

In [4]:
%ls
%cd workspace/LLaMA-Factory/
%pip install -r requirements.txt
%pip install -e .[torch,bitsandbytes]
%pip install autoawq --no-build-isolation
!GRADIO_SHARE=1 llamafactory-cli webui


CITATION.cff  README_zh.md  evaluation/        saves/    train_llama3.json
LICENSE       assets/       examples/          scripts/
MANIFEST.in   cache/        merge_llama3.json  setup.py
Makefile      data/         pyproject.toml     src/
README.md     docker/       requirements.txt   tests/
[Errno 2] No such file or directory: 'workspace/LLaMA-Factory/'
/workspace/LLaMA-Factory
Looking in indexes: http://mirrors.tencentyun.com/pypi/simple
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: http://mirrors.tencentyun.com/pypi/simple
Obtaining file:///workspace/LLaMA-Factory
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for llamafactory (pyproject.toml) ... done
  Created wheel for llamafactory: filename=llamafactory-0.9.4.dev0-0.editable-py3-none-any.whl size=28207 sha256=40

使用命令行微调模型
微调过程大约需要 30 分钟。


In [5]:
import json

args = dict(
  stage="sft",                                               # 进行指令监督微调
  do_train=True,
  model_name_or_path="Qwen/Qwen1.5-1.8B-Chat-AWQ", 			 # 使用 Qwen/Qwen1.5-1.8B-Chat-AWQ 模型
  dataset="identity,alpaca_en_demo",         				 # 使用 alpaca 和自我认知数据集
  template="qwen",                                           # 使用 llama3 提示词模板
  finetuning_type="lora",                                    # 使用 LoRA 适配器来节省显存
  lora_target="all",                                         # 添加 LoRA 适配器至全部线性层
  output_dir="qwen_lora",                                  # 保存 LoRA 适配器的路径
  per_device_train_batch_size=2,                             # 批处理大小
  gradient_accumulation_steps=4,                             # 梯度累积步数
  lr_scheduler_type="cosine",                                # 使用余弦学习率退火算法
  logging_steps=5,                                           # 每 5 步输出一个记录
  warmup_ratio=0.1,                                          # 使用预热学习率
  save_steps=1000,                                           # 每 1000 步保存一个检查点
  learning_rate=5e-5,                                        # 学习率大小
  num_train_epochs=3.0,                                      # 训练轮数
  max_samples=1000,                                          # 使用每个数据集中的 1000 条样本
  max_grad_norm=1.0,                                         # 将梯度范数裁剪至 1.0
  loraplus_lr_ratio=16.0,                                    # 使用 LoRA+ 算法并设置 lambda=16.0
  fp16=True,                                                 # 使用 float16 混合精度训练
  report_to="none",                                          # 关闭 wandb 记录器
  dataset_dir="data"
)

json.dump(args, open("train_qwen15.json", "w", encoding="utf-8"), indent=2)

%ls

!llamafactory-cli train train_qwen15.json

CITATION.cff  README_zh.md  evaluation/        saves/    train_llama3.json
LICENSE       assets/       examples/          scripts/  train_qwen15.json
MANIFEST.in   cache/        merge_llama3.json  setup.py
Makefile      data/         pyproject.toml     src/
README.md     docker/       requirements.txt   tests/
[INFO|2025-09-12 10:29:13] llamafactory.hparams.parser:414 >> Process rank: 0, world size: 1, device: cuda:0, distributed training: False, compute dtype: torch.float16
[INFO|tokenization_utils_base.py:2067] 2025-09-12 10:29:13,969 >> loading file vocab.json from cache at /root/.cache/huggingface/hub/models--Qwen--Qwen1.5-1.8B-Chat-AWQ/snapshots/eb196b26ac14d8bcf4144f9a3ab77b10ae77fe8f/vocab.json
[INFO|tokenization_utils_base.py:2067] 2025-09-12 10:29:13,969 >> loading file merges.txt from cache at /root/.cache/huggingface/hub/models--Qwen--Qwen1.5-1.8B-Chat-AWQ/snapshots/eb196b26ac14d8bcf4144f9a3ab77b10ae77fe8f/merges.txt
[INFO|tokenization_utils_base.py:2067] 2025-09-12 10:29:13

In [6]:
from llamafactory.chat import ChatModel
from llamafactory.extras.misc import torch_gc
%ls
%cd workspace/LLaMA-Factory/

args = dict(
  model_name_or_path="Qwen/Qwen1.5-1.8B-Chat-AWQ", # 使用 4 比特量化版 Llama-3-8b-Instruct 模型
  adapter_name_or_path="qwen_lora",                        # 加载之前保存的 LoRA 适配器
  template="qwen",                                         # 和训练保持一致
  finetuning_type="lora",                                    # 和训练保持一致
)
chat_model = ChatModel(args)

messages = []
print("使用 `clear` 清除对话历史，使用 `exit` 退出程序。")
while True:
  query = input("\nUser: ")
  if query.strip() == "exit":
    break
  if query.strip() == "clear":
    messages = []
    torch_gc()
    print("对话历史已清除")
    continue

  messages.append({"role": "user", "content": query})
  print("Assistant: ", end="", flush=True)

  response = ""
  for new_text in chat_model.stream_chat(messages):
    print(new_text, end="", flush=True)
    response += new_text
  print()
  messages.append({"role": "assistant", "content": response})

torch_gc()

/root/miniforge3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CITATION.cff  assets/      merge_llama3.json  src/
LICENSE       cache/       pyproject.toml     tests/
MANIFEST.in   data/        qwen_lora/         train_llama3.json
Makefile      docker/      requirements.txt   train_qwen15.json
README.md     evaluation/  scripts/
README_zh.md  examples/    setup.py
[Errno 2] No such file or directory: 'workspace/LLaMA-Factory/'
/workspace/LLaMA-Factory


[INFO|tokenization_utils_base.py:2067] 2025-09-12 11:08:17,209 >> loading file vocab.json from cache at /root/.cache/huggingface/hub/models--Qwen--Qwen1.5-1.8B-Chat-AWQ/snapshots/eb196b26ac14d8bcf4144f9a3ab77b10ae77fe8f/vocab.json
[INFO|tokenization_utils_base.py:2067] 2025-09-12 11:08:17,210 >> loading file merges.txt from cache at /root/.cache/huggingface/hub/models--Qwen--Qwen1.5-1.8B-Chat-AWQ/snapshots/eb196b26ac14d8bcf4144f9a3ab77b10ae77fe8f/merges.txt
[INFO|tokenization_utils_base.py:2067] 2025-09-12 11:08:17,211 >> loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--Qwen--Qwen1.5-1.8B-Chat-AWQ/snapshots/eb196b26ac14d8bcf4144f9a3ab77b10ae77fe8f/tokenizer.json
[INFO|tokenization_utils_base.py:2067] 2025-09-12 11:08:17,211 >> loading file added_tokens.json from cache at None
[INFO|tokenization_utils_base.py:2067] 2025-09-12 11:08:17,212 >> loading file special_tokens_map.json from cache at None
[INFO|tokenization_utils_base.py:2067] 2025-09-12 11:08:17,21

[INFO|2025-09-12 11:08:20] llamafactory.model.model_utils.quantization:143 >> Loading 4-bit AWQ-quantized model.
[INFO|2025-09-12 11:08:20] llamafactory.model.model_utils.kv_cache:143 >> KV cache is enabled for faster generation.


[INFO|auto.py:234] 2025-09-12 11:08:20,503 >> 
[INFO|modeling_utils.py:1308] 2025-09-12 11:08:20,504 >> loading weights file model.safetensors from cache at /root/.cache/huggingface/hub/models--Qwen--Qwen1.5-1.8B-Chat-AWQ/snapshots/eb196b26ac14d8bcf4144f9a3ab77b10ae77fe8f/model.safetensors
[INFO|modeling_utils.py:2411] 2025-09-12 11:08:20,506 >> Instantiating Qwen2ForCausalLM model under default dtype torch.float16.
[INFO|configuration_utils.py:1098] 2025-09-12 11:08:20,509 >> Generate config GenerationConfig {
  "bos_token_id": 151643,
  "eos_token_id": 151645
}

/root/miniforge3/lib/python3.10/site-packages/awq/__init__.py:21: DeprecationWarning: 
I have left this message as the final dev message to help you transition.

Important Notice:
- AutoAWQ is officially deprecated and will no longer be maintained.
- The last tested configuration used Torch 2.6.0 and Transformers 4.51.3.
- If future versions of Transformers break AutoAWQ compatibility, please report the issue to the Transform

[INFO|2025-09-12 11:08:22] llamafactory.model.model_utils.attention:143 >> Using torch SDPA for faster training and inference.
[INFO|2025-09-12 11:08:23] llamafactory.model.adapter:143 >> Loaded adapter(s): qwen_lora
[INFO|2025-09-12 11:08:23] llamafactory.model.loader:143 >> all params: 629,925,888
使用 `clear` 清除对话历史，使用 `exit` 退出程序。
Assistant: 您好！我是 小安，一个由 HolmeHome 制作的 AI 助手，很高兴见到您。有什么可以帮助您的吗？
Assistant: 当然，认知行为疗法（Cognitive Behavioral Therapy, CBT）是一种常用的心理治疗方法，它的基本理念是，人们的思想、情绪和行为是相互影响的。通过识别和改变这些负面的思维模式和行为习惯，可以有效地治疗许多心理问题，包括焦虑、抑郁、强迫症和创伤后应激障碍等。
Assistant: 北京的房价因地区、地段、楼层等因素而异，一般来说，市中心的房价较高，而靠近公园或学校的房价可能相对较低。近年来，由于调控政策的影响，北京的房价有所下降，但仍高于其他城市。具体房价情况建议您咨询当地的房地产中介或使用专业的房价比较工具来获取更准确的信息。
Assistant: 小米是一家中国公司，它的股票价格受多种因素影响，包括但不限于公司的业绩、行业动态、宏观经济形势、政策变化等。作为一个 AI 助手，我无法预测小米股票的未来走势，但通常情况下，股票价格受市场情绪和供求关系的影响，小米股票的表现会受到这些多方面因素的影响。建议您关注相关的财经新闻和数据，以及专业的投资顾问的建议，以做出更明智的投资决策。
Assistant: 作为 小安，我是一个由 HolmeHome 制作的 AI 助手，旨在为用户提供智能化的回答和帮助。请问您有什么问题或需要我提供的信息吗？


In [ ]:
import json

args = dict(
  stage="sft",                                               # 进行指令监督微调
  do_train=True,
  model_name_or_path="unsloth/llama-3-8b-Instruct-bnb-4bit", # 使用 4 比特量化版 Llama-3-8b-Instruct 模型
  dataset="identity,alpaca_en_demo,alpaca_zh_demo",          # 使用 alpaca 和自我认知数据集
  template="llama3",                                         # 使用 llama3 提示词模板
  finetuning_type="lora",                                    # 使用 LoRA 适配器来节省显存
  lora_target="all",                                         # 添加 LoRA 适配器至全部线性层
  output_dir="llama3_lora",                                  # 保存 LoRA 适配器的路径
  per_device_train_batch_size=2,                             # 批处理大小
  gradient_accumulation_steps=4,                             # 梯度累积步数
  lr_scheduler_type="cosine",                                # 使用余弦学习率退火算法
  logging_steps=5,                                           # 每 5 步输出一个记录
  warmup_ratio=0.1,                                          # 使用预热学习率
  save_steps=1000,                                           # 每 1000 步保存一个检查点
  learning_rate=5e-5,                                        # 学习率大小
  num_train_epochs=3.0,                                      # 训练轮数
  max_samples=300,                                           # 使用每个数据集中的 300 条样本
  max_grad_norm=1.0,                                         # 将梯度范数裁剪至 1.0
  loraplus_lr_ratio=16.0,                                    # 使用 LoRA+ 算法并设置 lambda=16.0
  fp16=True,                                                 # 使用 float16 混合精度训练
  report_to="none",                                          # 关闭 wandb 记录器
)

json.dump(args, open("train_llama3.json", "w", encoding="utf-8"), indent=2)

%ls

!llamafactory-cli train train_llama3.json

模型推理

In [ ]:
from llamafactory.chat import ChatModel
from llamafactory.extras.misc import torch_gc
%ls
%cd workspace/LLaMA-Factory/

args = dict(
  model_name_or_path="unsloth/llama-3-8b-Instruct-bnb-4bit", # 使用 4 比特量化版 Llama-3-8b-Instruct 模型
  adapter_name_or_path="llama3_lora",                        # 加载之前保存的 LoRA 适配器
  template="llama3",                                         # 和训练保持一致
  finetuning_type="lora",                                    # 和训练保持一致
)
chat_model = ChatModel(args)

messages = []
print("使用 `clear` 清除对话历史，使用 `exit` 退出程序。")
while True:
  query = input("\nUser: ")
  if query.strip() == "exit":
    break
  if query.strip() == "clear":
    messages = []
    torch_gc()
    print("对话历史已清除")
    continue

  messages.append({"role": "user", "content": query})
  print("Assistant: ", end="", flush=True)

  response = ""
  for new_text in chat_model.stream_chat(messages):
    print(new_text, end="", flush=True)
    response += new_text
  print()
  messages.append({"role": "assistant", "content": response})

torch_gc()

合并 LoRA 权重并上传模型
注意：Colab 免费版仅提供了 12GB 系统内存，而合并 8B 模型的 LoRA 权重需要至少 18GB 系统内存，因此你 无法 在免费版运行此功能。

In [2]:
# !hf auth login
from huggingface_hub import login

# 在 import transformers 之前登录
login(token="")  # 替换为你的真实 Token

# from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained(
#     "meta-llama/Meta-Llama-3-8B-Instruct",
#     token=True  # 自动使用已登录的 token
# )
# print("✅ Tokenizer 加载成功！")

/root/miniforge3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import json

args = dict(
  # model_name_or_path="meta-llama/Meta-Llama-3-8B-Instruct", # 使用非量化的官方 Llama-3-8B-Instruct 模型
  model_name_or_path="Qwen/Qwen1.5-1.8B-Chat", # 使用非量化的官方 Llama-3-8B-Instruct 模型
  adapter_name_or_path="qwen_lora",                       # 加载之前保存的 LoRA 适配器
  template="qwen",                                        # 和训练保持一致
  finetuning_type="lora",                                   # 和训练保持一致
  export_dir="qwen_lora_merged",                          # 合并后模型的保存目录
  export_size=2,                                            # 合并后模型每个权重文件的大小（单位：GB）
  export_device="auto",                                      # 合并模型使用的设备：`cpu` 或 `auto`
  export_hub_model_id="holmes0101/Qwen1.5-1.8B-Chat-AWQ_xiaoan",               # 用于上传模型的 HuggingFace 模型 ID
)

json.dump(args, open("merge_qwen15.json", "w", encoding="utf-8"), indent=2)

%cd /workspace/LLaMA-Factory/
!llamafactory-cli export merge_qwen15.json

/workspace/LLaMA-Factory
tokenizer_config.json: 1.29kB [00:00, 5.50MB/s]
vocab.json: 2.78MB [00:00, 10.7MB/s]
merges.txt: 1.67MB [00:00, 9.78MB/s]
tokenizer.json: 7.03MB [00:00, 17.4MB/s]
[INFO|tokenization_utils_base.py:2067] 2025-09-12 11:43:51,809 >> loading file vocab.json from cache at /root/.cache/huggingface/hub/models--Qwen--Qwen1.5-1.8B-Chat/snapshots/e482ee3f73c375a627a16fdf66fd0c8279743ca6/vocab.json
[INFO|tokenization_utils_base.py:2067] 2025-09-12 11:43:51,809 >> loading file merges.txt from cache at /root/.cache/huggingface/hub/models--Qwen--Qwen1.5-1.8B-Chat/snapshots/e482ee3f73c375a627a16fdf66fd0c8279743ca6/merges.txt
[INFO|tokenization_utils_base.py:2067] 2025-09-12 11:43:51,809 >> loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--Qwen--Qwen1.5-1.8B-Chat/snapshots/e482ee3f73c375a627a16fdf66fd0c8279743ca6/tokenizer.json
[INFO|tokenization_utils_base.py:2067] 2025-09-12 11:43:51,809 >> loading file added_tokens.json from cache at None
[INFO|t

In [ ]:
import json

args = dict(
  # model_name_or_path="meta-llama/Meta-Llama-3-8B-Instruct", # 使用非量化的官方 Llama-3-8B-Instruct 模型
  model_name_or_path="mlabonne/Meta-Llama-3.1-8B-Instruct-abliterated", # 使用非量化的官方 Llama-3-8B-Instruct 模型
  adapter_name_or_path="llama3_lora",                       # 加载之前保存的 LoRA 适配器
  template="llama3",                                        # 和训练保持一致
  finetuning_type="lora",                                   # 和训练保持一致
  export_dir="llama3_lora_merged",                          # 合并后模型的保存目录
  export_size=2,                                            # 合并后模型每个权重文件的大小（单位：GB）
  export_device="cpu",                                      # 合并模型使用的设备：`cpu` 或 `auto`
  export_hub_model_id="holmes0101/llama-3-8b-Instruct-bnb-4bit_xiaoan",               # 用于上传模型的 HuggingFace 模型 ID
)

json.dump(args, open("merge_llama3.json", "w", encoding="utf-8"), indent=2)

%cd /workspace/LLaMA-Factory/

!llamafactory-cli export merge_llama3.json